In [1]:
#!/usr/bin/env python3

########################################################################
# File: Chan_Nicholas_DAG.ipynb
#Purpose:
# main(infile='FILE_PATH',outfile='FILE_PATH')
#
# Author: Nicholas Chan
# History: 11/08/2021 Created
########################################################################

# Assignment 4: Problem 15
<br>
For this assignment, we were to find the longest path between two nodes (start and end) in an edge-weighted DAG. <br>
We were given data consisting of the start node key, stop node key, and an adjacency list with edge weights. To <br>
solve this assignment, I created an Edge class, Node class, and Graph class to represent a DAG. Graph objects <br>
are composed of Node objects connected by Edge objects. A topological sort algorithm in the form of a method in <br>
the Graph Class finds the highest scoring path between the start and end nodes of a DAG.

# Edge Class
<br>
The Edge Class is used to create objects mirroring the utility of edges in the real graph. Edges are mainly <br>
used to represent relationships between two nodes given their keys and corresponding edge weight. 

In [2]:
class Edge:
    '''
    Edge class is used to create objects that store data on the relationship
    between nodes held in a DAG. DAGs are represented by Edges and Nodes in
    a Graph object.
    '''
    def __init__(self, src:'int', dest:'int', weight:'int'):
        '''
        Edge objects are initialized with three main fields, each of which
        contain an int as data. Edge objects store infomation on the key of 
        their source nodes and end nodes. Edges also store the edge weight 
        between a source and sink node.
        '''
        self.src = src
        self.dest = dest
        self.weight = weight
    
    def printEdge(self):
        ''' Prints out a representation of the current Edge object '''
        print(f"[src: {self.src} | dest: {self.dest}]")
        print(f"weight: {self.weight}")
        print()
        
    def pstring(self):
        ''' Returns a string representation of the current Edge object '''
        return f"({self.src}->{self.dest}:{self.weight})" 

# Node Class
<br>
The Node Class is used to create objects mirroring the utility of Nodes in the real graph. Nodes are mainly <br>
used to represent the vertices of a Graph and hold information such as a key, inEdges, outEdges, and an accumulated <br>
score which may be used to determine the priority cetain paths may take. 

In [3]:
class Node:
    '''
    Node class is used to create objects to populate DAGs represented by 
    Graph objects. Nodes can be connected by Edge objects in a Graph object.
    '''
    def __init__(self, key:'int', score=float("-inf")):
        '''
        Nodes are initialized with their respective key. Nodes are given
        inEdges and outEdges after a list of edges are fed into a Graph 
        object. Graph objects will initialize Nodes with inEdges and 
        outEdges corresponding to their given edgeLists. During topological
        sorting in a Graph object, nodes are given scores based on their 
        location in a path of nodes and edges.
        '''
        self.key = key
        self.inEdges = [] 
        self.outEdges = []
        # Following attributes are for tracking score and are populated during topSort
        self.scoreEdgeList = [] # List holds inEdges of Node that have been evaluated
        self.maxEdge = None # maxEdge will hold the edge yielding the highest score for this Node
        self.tmpScore = 0 # tmpScore is a temporary variable for holding the score associated with maxEdge
        self.score = score # score is set to tmpScore's value after a Node has been fully evaluated, initialized to -inf 
    
    def addInEdge(self, inEdge):
        ''' Small method for adding an inEdge to a Node's list of inEdges '''
        self.inEdges.append(inEdge)
        
    def addOutEdge(self, outEdge):
        ''' Small method for adding an outEdge to a Node's list of outEdges '''
        self.outEdges.append(outEdge)   

    def printNode(self):
        ''' Prints information held by the Node. '''
        print(f"key: {self.key}")
        print(f"inEdges: {[e.pstring() for e in self.inEdges]}")
        print(f"outEdges: {[e.pstring() for e in self.outEdges]}")
        print(f"score: {self.score}")
        print()

# Graph Class
<br>
The Graph Class is used to create objects mirroring the utility real graphs. Graphs are mainly <br>
used to represent a network of vertices (Nodes) connected by relationships (Edges). The weights between <br>
vertices can be used to calculate a score which can be accumulated through nodes and edges in a graph. <br>
A path in a graph is a sequence of edges which join vertices. Paths can also be characterized by the  <br>
previously mentioned accumulation of scores. The type of graph that we wish to represent with the Graph <br>
class is a DAG. To find the longest path in a DAG, a topological sort must be performed to fully evaluate <br>
a DAG's paths. Eventually through a topological sort, we may determine the longest path running from a DAG's  <br>
start and end nodes.

In [4]:
class Graph:
    ''' 
    Graph class is used to create objects for representing DAGs. Graph class uses 
    Nodes connected by Edges to represent a DAG. Graph objects perform topological
    sorts on the DAGs which they represent and contain information on the highest
    scoring path from start node to end node as well as such a path's associated score.
    '''
    def __init__(self, start:'int', end:'int', edges:'list[Edge]'):
        '''
        init method of Graph class initializes a graph with nodes connected by edges 
        given information on the start node, end node, and list of edges of a DAG. 
        Graph objects are initialized with their topologies evaluated. Final paths with 
        their associated scores are also available, but only after the finder method 
        has been called.
        '''
        self.start = start # Key of start node
        self.end = end # Key of end node
        self.edges = edges # List of edges to be used to generate nodes and node relationships
        self.edgeHolder = self.edges.copy() # Copy of edges list that is used to track remaining edges when topologically sorting
        self.nodes = dict() # Dict of nodes in graph
        for e in edges: # Following for loop initializes all possible nodes in the dict called nodes
            if e.src not in self.nodes:
                self.nodes[e.src] = Node(key=e.src)
            if e.dest not in self.nodes:
                self.nodes[e.dest] = Node(key=e.dest)
        self.nodePopulate() # nodePopulate method populates Graph object's Nodes with inEdges and outEdges
        self.topOrder = self.topOrder() # topOrder method solves topology of DAG represented by Graph object and stores it as a list
        self.finalScore = 0 # finalScore is the total score recorded for a path from start to end, must call finder method to populate finalScore
        self.finalPath = [] # finalPath is the final path from start to end, must call finder method to populate finalPath
            
    def nodePopulate(self):
        '''
        Creates node relationships
        MUST CALL THIS BEFORE topOrder
        '''
        for e in self.edges:
            self.nodes[e.src].addOutEdge(e)
            self.nodes[e.dest].addInEdge(e)
        # Remove all inEdges to start and outEdges from end in case
        self.nodes[self.start].inEdges = []
        self.nodes[self.end].outEdges = []
    
    def topOrder(self):
        '''
        Keep a list of nodes in the order to solve them. 
        Constantly update set of candidate nodes with no incoming edges
        '''
        List = [] # List of NODES
        noInEdgeNodes = {n for n in self.nodes.values() if len(n.inEdges) == 0} # Nodes with no inEdges
        for n in noInEdgeNodes:
            if n.key != self.start:
                n.tmpScore = float("-inf") # Initialize nodes with -inf tmpScores
        candidates = noInEdgeNodes.copy() 
        while len(candidates) > 0: # 
            node = candidates.pop()
            List.append(node)
            if node not in noInEdgeNodes: # Node originally had inEdges, and has fully populated scoreEdgeList
                node.maxEdge = max(node.scoreEdgeList, key=lambda x:x[0].weight + x[1])
                node.score = node.maxEdge[0].weight + node.maxEdge[1]
                node.tmpScore = node.score
            if len(node.outEdges) == 0:
                continue
            for outEdge in node.outEdges: # For each outedge from curr node, remove outEdge from nextNode's inEdges and add curr Edge to nextNode's scoreEdgeList
                if outEdge.dest == self.start: # Catches case where a node has an outEdge to the startNode, which shouldn't happen in a DAG
                    self.edgeHolder.remove(outEdge)
                    continue
                nextNode = self.nodes[outEdge.dest]
                nextNode.inEdges.remove(outEdge) # Remove curr's outedge from next's inEdges
                self.edgeHolder.remove(outEdge)
                nextNode.scoreEdgeList.append((outEdge,node.tmpScore))
                if len(nextNode.inEdges) == 0: # If nextNode is ready to become a candidate
                    candidates.add(nextNode)                    
        if len(self.edgeHolder) > 0:
            return "Input graph not a DAG"
        else:
            # Should return a list of node objects that represent the topological order of the graph
            return List

    def finder(self):
        '''
        We only need order (Topologically sorted node list) for its final element.
        Method finds path from source node to end node.
        '''
        stack = []
        lastEnd = self.nodes[self.end]
        while lastEnd.maxEdge != None:
            stack.append(self.nodes[lastEnd.maxEdge[0].dest].key)
            lastEnd = self.nodes[lastEnd.maxEdge[0].src]
        stack.append(lastEnd.key)
        self.finalScore = self.nodes[self.end].score
        self.finalPath = stack[::-1]
        return (self.nodes[self.end].score, stack[::-1])        
    
    def printScore(self):
        ''' 
        Small method for returning the score of the highest scoring 
        path from start node to end node in a DAG.
        '''
        if isinstance(self.topOrder, str):
            return self.topOrder
        else:
            return self.finalScore
    
    def printPath(self):
        ''' 
        Small method for returning the highest scoring path from start 
        node to end node in a DAG as a string.
        '''
        if isinstance(self.topOrder, str):
            return
        pathway = ""
        for key in self.finalPath:
            pathway += str(key)
            if self.finalPath.index(key) < len(self.finalPath) -1:
                pathway += "->"
        return pathway

# Main function 
Main function is written here. This function handles argument parsing, input parsing, and output.

In [5]:
def main(infile, outfile='', inCL=None):
    '''
    main function parses in data specified by infile and creates 
    a list of Edge objects from given data. The finished Edge List,
    start node information, and end node information are used to initialize
    a Graph object. The Graph method finder is called to compute the final
    score and final path from start node to end node of a DAG. Latter half 
    of main function handles printing/writing of output.
    '''
    edgeList = []
    with open(infile,'r') as myfile:
        startNode = int(next(myfile))
        endNode = int(next(myfile))
        for line in myfile:
            line = line.replace(':', '->')
            myLine = line.rstrip().split('->')
            edgeData = [int(i) for i in myLine]
            e = Edge(edgeData[0], edgeData[1], edgeData[2])
            edgeList.append(e)
    # INITIALIZE GRAPH GIVEN START, END, AND LIST OF EDGES (ADJ LIST)
    myGraph = Graph(start = startNode, end = endNode, edges = edgeList)
    myGraph.finder()
    
    print(myGraph.printScore()) # Prints out final score of path from source to sink nodes
    if myGraph.printPath() != None: # printPath() will return None if graph is not a DAG
        print(myGraph.printPath()) # Prints out final path from source to sink nodes
    if len(outfile) > 0: # PROCEED with writing to output file if outfile is specified
        with open(outfile, 'w') as myfile:
            myfile.write(str(myGraph.printScore()) + '\n') # Writes out final score of path from source to sink nodes
            if myGraph.printPath() != None: # printPath() will return None if graph is not a DAG
                myfile.write(myGraph.printPath())  # Writes out final path from source to sink nodes


In [6]:
if __name__ == "__main__":
    main(infile = 'rosalind_ba5d.txt', outfile = 'out.txt')

57
3->13->20->23


In [7]:
# INSPECTION

# INSPECTION TEAM
# Jodi Lee
# Nabil Mohammed
# Richard Yong
# Hsiang-Yun Lu (Eloise)

# RESPONSES
# - Work on implementing main function
# - Write more markdown comments
# - Use float(-inf) instead of -999 for initializing node scores
# - Clean up code
# - Fix camel casing on some variables
# - Add more docstrings and inline comments

# CORRECTIONS
# - Finished main function
# - Wrote markdown comments
# - Initialized node scores with float('-inf')
# - Cleaned code
# - Fixed camel casing of variables
# - Added more docstrings and inline comments